In [2]:
import pandas as pd

donations_df = pd.read_excel('./data/ForeignGifts.xls', skiprows=1)
# donations_df.loc[:, 'Foreign Gift Amount'] = donations_df.loc[:, 'Foreign Gift Amount'].apply('{:,}'.format)
donations_df.head()

,ID,OPEID,Institution Name,City,State,Foreign Gift Received Date,Foreign Gift Amount,Gift Type,Country of Giftor,Giftor Name
0,1,324200,Carnegie Mellon University,Pittsburgh,PA,2015-01-01,77391377,Contract,QATAR,Qatar Foundation for Education
1,2,143400,American University (The),Washington,DC,2012-01-25,500000,Monetary Gift,BAHRAIN,Kingdom of Bahrain
2,3,143400,American University (The),Washington,DC,2013-05-28,500000,Monetary Gift,BAHRAIN,Sheikh Salman Bin Al-Khalifa
3,4,143400,American University (The),Washington,DC,2015-09-10,500000,Monetary Gift,KUWAIT,NaN
4,5,143400,American University (The),Washington,DC,2012-05-17,610000,Monetary Gift,ENGLAND,NaN


In [3]:
donations_df.groupby('Institution Name').count()['Foreign Gift Amount'].sort_values(ascending=False)

schools = donations_df['Institution Name'].unique()
school_lst = []
for school in schools:
    school_lst.append(school.split(' (')[0])

In [4]:
import time
import numpy as np
from geopy.geocoders import Nominatim
from geopy import exc


geolocator = Nominatim(user_agent='foerign_donations')
schools_df = pd.DataFrame(columns=['Latitude', 'Longitude'])
school_names_to_fix = []
for school in school_lst:
    try:
        location = geolocator.geocode(school)
        if location.address.split(' ')[-1].strip() != 'America':
            school_names_to_fix.append(school)
            print(school + '\t' + 'SKIPPED')
            continue
        schools_df.at[school, 'Latitude'] = location.latitude
        schools_df.at[school, 'Longitude'] = location.longitude
        print(school + '\t' + location.address.split(' ')[-1])
    except AttributeError:
        schools_df.at[school, 'Latitude'] = np.nan
        schools_df.at[school, 'Longitude'] = np.nan
        school_names_to_fix.append(school)
        print(school + '\t' + 'SKIPPED')
    except exc.GeocoderTimedOut:
        print('Paused... Waiting for server.')
        time.sleep(10)
        location = geolocator.geocode(school)
        schools_df.at[school, 'Latitude'] = location.latitude
        schools_df.at[school, 'Longitude'] = location.longitude        
    time.sleep(1)

Carnegie Mellon University	America
American University	America
Amherst College	America
Arizona State University	America
Michigan State University	America
University of Colorado Boulder	America
Massachusetts Institute of Technology	America
Virginia Commonwealth University	America
California Institute of Technology	America
University of Arkansas	America
Georgetown University	America
Harvard University	America
Northwestern University	America
University of California, San Diego	America
Duke University	America
Texas A&M University	America
New York University	America
Hult International Business School	America
University of Illinois at Urbana-Champaign	America
Rice University	America
Kansas State University	America
University of Southern California	America
University of Arizona	America
Yale University	America
Johns Hopkins University	SKIPPED
University of California, Los Angeles	America
Eastern Washington University	America
George Washington University	America
University of Washington - Seatt

In [5]:
school_names_to_fix

['Johns Hopkins University',
 'New York Institute of Technology',
 'Teachers College, Columbia University',
 'California Institute of Advanced Management',
 'Indiana University - Purdue University Indianapolis',
 'University of North Texas Health Science Center at Fort Worth',
 'Rochester Institute of Technology',
 'University of Vermont and State Agricultural College',
 'Iowa State University of Science & Technology',
 'Polytechnic Institute of New York University',
 'Keck Graduate Institute of Applied Life Sciences',
 'St. Lawrence University',
 'Valencia College',
 'Western International University']

In [6]:
new_school_names = {'Johns Hopkis University': 'Johns Hopkins University, USA',
                    'New York Institute of Technology': 'New York Institute of Technology, USA',
                    'Teachers College, Columbia University': 'Teachers College',
                    'California Institute of Advanced Management': 'Platt College Los Angeles',
                    'Indiana University - Purdue University Indianapolis': 'IUPUI',
                    'University of North Texas Health Science Center at Fort Worth': 'UNTHSC',
                    'Rochester Institute of Technology': 'RIT, USA',
                    'University of Vermont and State Agricultural College': 'University of Vermont',
                    'Iowa State University of Science & Technology': 'Iowa State University',
                    'Polytechnic Institute of New York University': 'New York City College of Technology',
                    'Keck Graduate Institute of Applied Life Sciences': 'Keck Graduate Institute',
                    'St. Lawrence University': 'Saint Lawrence University, USA',
                    'Valencia College': 'Valencia College West Campus',
                    'Western International University': 'Fountainhead Pkwy, Arizona'
                   }

for school, hack_name in new_school_names.items():
    try:
        location = geolocator.geocode(hack_name)
        schools_df.at[school, 'Latitude'] = location.latitude
        schools_df.at[school, 'Longitude'] = location.longitude
        print(school + '\t' + hack_name + '\t' + location.address + '\n')
    except AttributeError:
        schools_df.at[school, 'Latitude'] = np.nan
        schools_df.at[school, 'Longitude'] = np.nan
        school_names_to_fix.append(school)
    except exc.GeocoderTimedOut:
        time.sleep(10)
        schools_df.at[school, 'Latitude'] = location.latitude
        schools_df.at[school, 'Longitude'] = location.longitude        
    time.sleep(1)

Johns Hopkis University	Johns Hopkins University, USA	Johns Hopkins University, Cresmont Avenue, Remington, Baltimore, Maryland, 21211, United States of America

New York Institute of Technology	New York Institute of Technology, USA	New York Institute of Technology, Polo Drive, Wheatley, Old Westbury, Nassau County, New York, 11545, United States of America

Teachers College, Columbia University	Teachers College	Teachers College, West 122nd Street, Morningside Heights, Manhattan Community Board 9, New York County, New York, 10027, United States of America

California Institute of Advanced Management	Platt College Los Angeles	Platt College - Los Angeles, Fremont Avenue, Ramona Park, Alhambra, Los Angeles County, California, 91803, United States of America

Indiana University - Purdue University Indianapolis	IUPUI	Rotary Park Space at IUPUI, Glendale, Indianapolis, Marion, Indiana, United States of America

University of North Texas Health Science Center at Fort Worth	UNTHSC	855, Montgom

In [7]:
schools_df.isnull().values.any()

False

In [8]:
school_names_map = {}
for split_name, orig_name in zip(schools_df.index, schools):
    school_names_map[split_name] = orig_name

schools_df_final = schools_df.rename(school_names_map)

In [9]:
merged_df = pd.read_csv('donation_democracy_index.csv')

In [10]:
country_lst = list(merged_df.groupby('Country of Giftor').count().index)
country_df = pd.DataFrame(index=country_lst, columns=['Latitude', 'Longitude'])
country_name_to_fix = []
for country in country_lst:
    try:
        location = geolocator.geocode(country)
        country_df.at[country, 'Latitude'] = location.latitude
        country_df.at[country, 'Longitude'] = location.longitude
    except AttributeError:
        country_df.at[country, 'Latitude'] = np.nan
        country_df.at[country, 'Longitude'] = np.nan
        country_name_to_fix.append(country)
    except exc.GeocoderTimedOut:
        time.sleep(10)
        country_df.at[country, 'Latitude'] = location.latitude
        country_df.at[country, 'Longitude'] = location.longitude
    print(country, location.latitude, location.longitude)
    time.sleep(1)  

Afghanistan 33.7680065 66.2385139
Angola -11.8775768 17.5691241
Argentina -34.9964963 -64.9672817
Armenia 40.7696272 44.6736646
Australia -24.7761086 134.755
Austria 47.2000338 13.199959
Azerbaijan 40.3936294 47.7872508
Bahamas 24.7736546 -78.0000547
Bahrain 26.1551249 50.5344606
Bangladesh 24.4768783 90.2932426
Barbados 13.1500331 -59.5250305
Belarus 53.4250605 27.6971358
Belgium 50.6402809 4.6667145
Bermuda 32.3018217 -64.7603583
Bolivia -17.0568696 -64.9912286
Bosnia And Herzegovina 44.3053476 17.5961467
Bosnia-Hercegovina 44.3053476 17.5961467
Botswana -23.1681782 24.5928742
Brazil -10.3333333 -53.2
British West Indies 21.772793399999998 -72.21725822003631
Bulgaria 42.6073975 25.4856617
Burma 17.1750495 95.9999652
Canada 61.0666922 -107.9917071
Cayman Islands (The) 19.5417212 -80.5667132
Ceylon 43.5335672 -94.6316462
Chile -31.7613365 -71.3187697
China 35.000074 104.999927
Colombia 2.8894434 -73.783892
Costa Rica 10.2735633 -84.0739102
Croatia 45.5643442 17.0118954
Curaçao 12.1845 

In [11]:
merged_country_df = pd.merge(merged_df, country_df, left_on='Country of Giftor', right_index=True, how='outer')
merged_country_df = merged_country_df.rename({'Latitude': 'Country Latitude', 'Longitude': 'Country Longitude'}, axis=1)

In [12]:
merged_final_df = pd.merge(merged_country_df, schools_df_final, left_on='Institution Name', right_index=True, how='outer')
merged_final_df = merged_final_df.rename({'Latitude': 'School Latitude', 'Longitude': 'School Longitude'}, axis=1)
merged_final_df.to_csv('merged_data_w_coord.csv')

In [19]:
merged_final_df.columns

Index(['Country of Giftor', 'Institution Name', 'ID', 'OPEID', 'City', 'State',
       'Foreign Gift Received Date', 'Foreign Gift Amount', 'Gift Type',
       'Giftor Name', 'Country to merge on', 'Rank', 'Score',
       'Electoral process and pluralism', 'Functioning of government',
       'Political participation', 'Political culture', 'Civil liberties',
       'Regime type', 'Region', 'Country Latitude', 'Country Longitude',
       'School Latitude', 'School Longitude'],
      dtype='object')